In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/4.part'):
    shutil.rmtree('./file/4.part')

os.makedirs('./file/4.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part_code'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/3.part_code/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['Product No'] = input_['Product No'].astype(int)
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                url = f'''https://www.gates.com/us/en/search.p.{input_.loc[a, 'Product Number']}.v.{input_.loc[a, 'Part Code']}.html'''

                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.get(url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                
                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =
                
                json_ = json.loads(resp.text.split('new s7viewers.MixedMediaViewer(')[1].split('"localizedTexts": localisedTexts')[0].strip().removesuffix(',') + '}')

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//li[@class="nested-accordion__item nested-accordion__item--level1"]/div[@role="tabpanel"]/table[@class="table c-table c-table--split"]/tbody/tr')

                # = = = = = = = = = = = = = = =

                i, part_number, sku, title, upc, dict_information = 0, '', '', '', '', {}
                for tr in list_tr:
                    key, value = tr.xpath('./th/text()')[0].strip().removesuffix('#').strip(), tr.xpath('./td/text()')[0].strip() if tr.xpath('./td/text()') else ''
                    if key == 'Part':
                        part_number = value.strip()
                    elif key == 'Product':
                        sku = value.strip()
                    elif key == 'Description':
                        title = value.strip()
                    elif key == 'UPC':
                        upc = value.strip()
                    else:
                        dict_information[str(i)] = {key: value.strip()}
                        i += 1

                # = = = = = = = = = = = = = = =

                if not dict_information:
                    raise

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'Product No': input_.loc[a, 'Product No'],
                                         'Product Name': input_.loc[a, 'Product Name'],
                                         'Product Number': input_.loc[a, 'Product Number'],
                                         'No': input_.loc[a, 'No'],
                                         'Part Number': part_number,
                                         'Type': input_.loc[a, 'Type'],
                                         'SKU': sku,
                                         'UPC': upc,
                                         'Title': title,
                                         'Url': url,
                                         'Json_Information': json.dumps(dict_information),
                                         'Part Code': input_.loc[a, 'Part Code'],
                                         'Asset': json_['params']['asset'].strip()}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Product No']}.{input_.loc[a, 'Product Name']} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Code']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Product No', 'No'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/4.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Product No', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/4.part/{file}',
              f'''./file/4.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 23.7235-086372
[剩余数量：1000] - [当前时间：16:01:24]

[状态：ok，尝试次数：3] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 30.7235-086482
[剩余数量：999] - [当前时间：16:01:24]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 28.7235-086471
[剩余数量：998] - [当前时间：16:01:26]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 33.7236-003035
[剩余数量：997] - [当前时间：16:01:29]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 32.7236-003015
[剩余数量：996] - [当前时间：16:01:29]

[状态：ok，尝试次数：5] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 31.7235-086497
[剩余数量：995] - [当前时间：16:01:29]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 34.7236-003065
[剩余数量：994] - [当前时间：16:01:30]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Coupling

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3100.00it/s]

Done ~
